In [1]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (TextVectorization, Dense, MultiHeadAttention, LayerNormalization, 
                                     Layer, Embedding, Input, Dropout)
from tensorflow.keras.callbacks import EarlyStopping

2023-03-02 14:03:58.738911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 14:03:58.925704: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 14:03:58.962602: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-02 14:03:59.621151: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (TextVectorization, Dense, MultiHeadAttention, LayerNormalization, 
                                     Layer, Embedding, Input, Dropout)
from tensorflow.keras.callbacks import EarlyStopping

import logging

GPU_FROM = 0
GPU_TO = 1  

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs visible:{len(visible_devices)}")
tf.config.set_visible_devices(visible_devices[GPU_FROM:GPU_TO],'GPU')

visible_devices = tf.config.get_visible_devices('GPU')
logging.info(f"Num GPUs to be used: {len(visible_devices)}")

# Build dataset

In [3]:
FULL_VOCAB = 'abcdefghijklmnopqrstuvwxyz'
SEQ_LEN = 10

In [4]:
def dataset1(vocab_size=2, dataset_size=10000, seq_len=10, full_vocab=FULL_VOCAB):
  """
  Inserts a space after a the first character in the vocabulary (and nowhere else)
  """
  assert vocab_size > 1
  vocab = list(full_vocab[:vocab_size])
  inputs = np.random.choice(vocab, size=(dataset_size, seq_len))
  outputs = np.where(inputs == vocab[0], 1., 0.).astype(np.float32)  # 1 = space, 0 = no space
  concatenated_inputs = np.array([''.join(row) for row in inputs])
  return concatenated_inputs, outputs

def dataset2(vocab_size=2, dataset_size=10000, seq_len=10, full_vocab=FULL_VOCAB):
  """
  Inserts a space after the combination of 1st->2nd character in the vocabulary (and nowhere else)
  """
  assert vocab_size > 1
  vocab = list(full_vocab[:vocab_size])
  inputs = np.random.choice(vocab, size=(dataset_size, seq_len))
  outputs = np.zeros_like(inputs, dtype=np.float32)
  for i, example in enumerate(inputs):
    previous_char = example[0]
    for j, char in enumerate(example[1:]):
      if (previous_char == vocab[0]) and (char == vocab[1]):  # 1 = space, 0 = no space
        outputs[i, j+1] = 1.
      previous_char = char
  concatenated_inputs = np.array([''.join(row) for row in inputs])
  return concatenated_inputs, outputs

def dataset3(vocab_size=2, dataset_size=10000, seq_len=10, insert_space_every=3, full_vocab=FULL_VOCAB):
  """
  Inserts a space after a certain number of characters, no matter what the characters
  """
  assert vocab_size > 1
  vocab = list(full_vocab[:vocab_size])
  inputs = np.random.choice(vocab, size=(dataset_size, seq_len))
  outputs = np.zeros_like(inputs, dtype=np.float32)
  outputs[:, np.arange(insert_space_every-1, outputs.shape[1], insert_space_every)] = 1.
  concatenated_inputs = np.array([''.join(row) for row in inputs])
  return concatenated_inputs, outputs

In [5]:
DATASET_FN = dataset2

train_ds = tf.data.Dataset.from_tensor_slices(DATASET_FN(vocab_size=2, seq_len=SEQ_LEN))
valid_ds = tf.data.Dataset.from_tensor_slices(DATASET_FN(vocab_size=2, seq_len=SEQ_LEN))
test_ds = tf.data.Dataset.from_tensor_slices(DATASET_FN(vocab_size=2, seq_len=SEQ_LEN))
train_ds.element_spec

2023-03-02 14:04:01.366617: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 14:04:02.099400: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22296 MB memory:  -> device: 0, name: GeForce RTX 3090, pci bus id: 0000:3b:00.0, compute capability: 8.6


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(10,), dtype=tf.float32, name=None))

In [6]:
train_ds = train_ds.shuffle(1000).batch(128)
valid_ds = valid_ds.batch(128)
test_ds = test_ds.batch(128)

In [7]:
for test_inputs, test_outputs in train_ds.take(1):
  pass
print(test_inputs)
tf.print(test_outputs,summarize=-1)

tf.Tensor(
[b'abbbbabaab' b'bbbababbba' b'abbbaabbaa' b'abbbbbaaba' b'aaabaaabaa'
 b'aaabaaabaa' b'aaaabbaaba' b'abaabababb' b'bbabbbbbaa' b'aabaaaaaaa'
 b'bbbaabbabb' b'baababaaab' b'abbbbaaabb' b'abbbabbbbb' b'baababbbaa'
 b'abbaaababa' b'aaabbabaaa' b'abbbabaaab' b'bbbbbbbabb' b'baabababbb'
 b'abbbaabbab' b'ababababab' b'abbaaaabab' b'bbbbbababa' b'ababaabbbb'
 b'bbabbabbbb' b'abaaaabaaa' b'babaaaaaab' b'bbbaaabaaa' b'aaaaababba'
 b'babbababaa' b'ababaababa' b'aaaaabaaaa' b'baaabaaaba' b'aabbaaabba'
 b'aaaabbbaba' b'bbbaaabbaa' b'abababbbba' b'bababababb' b'aabbbaabab'
 b'ababbbbaab' b'baabaabaaa' b'ababaabbab' b'bbaaabbbbb' b'baaabbbaaa'
 b'abbbaabaaa' b'abbbbabaab' b'abbabbabbb' b'bbbaaaaaab' b'aabbababba'
 b'aaaaaabbba' b'baabbbbbba' b'baaabaaabb' b'baaaaaabba' b'aaabbaaaaa'
 b'aaaabbabba' b'aabaaabbba' b'abbbabbabb' b'bababbbaab' b'aabbbabbbb'
 b'babbabbaaa' b'bbaaaabbbb' b'abbbbbbaaa' b'abbbabaaab' b'baaabbaaba'
 b'bbabbaabab' b'babbbabbab' b'bbabbababb' b'baaabaabba' b'abaabaa

# Build layers

In [65]:
# # Embedding defn
def positional_encoding(length, depth):
    """
    Generates a matrix following:
    $$
        PE_{pos,i} = trig(\frac{pos, 10000^{\frac{i, d}})
    $$
    where d is the dimensionality of the output embedding and the position
    is defined absolutely (from 0).
    """
    per_trig_d_model = depth/2
    

    positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
    depths = np.arange(per_trig_d_model)[np.newaxis, :]/per_trig_d_model   # (1, depth/2)
    angle_rates = 1 / (10000**depths)         # (1, depth/2)
    angle_rads = positions * angle_rates      # (seq, depth/2)

    pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)], # (seq, depth)
      axis=-1) 

    return tf.cast(pos_encoding, dtype=tf.float32)

class PostionalEmbedding(tf.keras.layers.Layer):
    # TODO: FIX spelling
    def __init__(
            self,
            vocab_size, 
            d_model, 
            max_seq_len,
            pos_multiplier=1, 
            mask_zero=True):
        """
        Generate a layer to embed input tokens that are already int-encoded
         through a lookup embedding and positional information 
         (through additive positional embeddings as in Vaswani 2017).
        """
        super().__init__()
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=mask_zero) 
        self.pos_encoding = positional_encoding(length=max_seq_len, depth=d_model)
        self.supports_masking = True
        self.pos_multiplier = pos_multiplier

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x, mask=None):
        # Assumes (batch, seq_len) int-encoded inputs
        x = self.embedding(x) # (batch, seq_len, d_model)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # TODO: try running without this
        x = x + self.pos_multiplier*self.pos_encoding[tf.newaxis, :, :] # new axis for batch dimension - try without
        return x

def point_wise_feed_forward_network(
    d_model, # Input/output dimensionality.
    dff # Inner-layer dimensionality.
    ):

    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # Shape `(batch_size, seq_len, dff)`.
      tf.keras.layers.Dense(d_model)  # Shape `(batch_size, seq_len, d_model)`.
    ])

class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self,*,
               d_model, # Input/output dimensionality.
               embedding_dim,
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               dropout_rate=0.1,
               **kwargs
               ):
        super().__init__(**kwargs)
        self.supports_masking = True

        # Multi-head self-attention.
        self.mha = tf.keras.layers.MultiHeadAttention(
            num_heads=num_attention_heads,
            key_dim=d_model, # Size of each attention head for query Q and key K.
            dropout=dropout_rate, # TODO: maybe dropout?
            )
        # Point-wise feed-forward network.
        self.ffn = point_wise_feed_forward_network(embedding_dim, dff)

        # Layer normalization.
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # Dropout for the point-wise feed-forward network.
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, x, training, mask):

        # A boolean mask.
        if mask is not None:
            mask1 = mask[:, :, None] # (B, seq, 1)
            mask2 = mask[:, None, :] # (B, 1, seq)
            attention_mask = mask1 & mask2 
            # For each element in the sequence - what other elements can it attend to
            # only defined in this simple self-cartesion product because we are
            # simply attempting to mask away unused token slots for efficiency
            # Has block strucure, NOT upper triangular like a causal mask would
        else:
            attention_mask = None

        # Multi-head self-attention output (`tf.keras.layers.MultiHeadAttention `).
        attn_output = self.mha(
            query=x,  # Query Q tensor.
            value=x,  # Value V tensor.
            key=x,  # Key K tensor.
            attention_mask=attention_mask, # A boolean mask that prevents attention to certain positions.
            training=training, # A boolean indicating whether the layer should behave in training mode.
            )

        # Multi-head self-attention output after layer normalization and a residual/skip connection.
        out1 = self.layernorm1(x + attn_output)  # Shape `(batch_size, input_seq_len, d_model)`

        # Point-wise feed-forward network output.
        ffn_output = self.ffn(out1)  # Shape `(batch_size, input_seq_len, d_model)`
        # ffn_output = self.dropout1(ffn_output, training=training) # TODO: try removing dropout errywhere
        # Point-wise feed-forward network output after layer normalization and a residual skip connection.
        out2 = self.layernorm2(out1 + ffn_output)  # Shape `(batch_size, input_seq_len, d_model)`.

        return out2

class Encoder(tf.keras.layers.Layer):
    def __init__(self,
               *,
               num_layers,
               d_model, # Input/output dimensionality.
               num_attention_heads,
               embedding_dim, # Equivalent to kevin's d_model
               dff, # Inner-layer dimensionality.
               tokenizer, # int-mode tokenizer for input text,
               seq_len,
               dropout_rate=0.1,
               pos_multiplier=1 
               ):
        super().__init__()

        self.d_model = d_model # equivalent to Kevin's key dim
        self.num_layers = num_layers
        self.supports_masking = True
        # Tokenization
        self.tokenizer = tokenizer

        # Embeddings + Positional encoding
        self.pos_embedding = PostionalEmbedding(tokenizer.vocabulary_size(), embedding_dim, seq_len, pos_multiplier=pos_multiplier)

        # Encoder layers.
        print(d_model, embedding_dim, num_attention_heads, dff)
        self.enc_layers = [
            EncoderLayer(
              d_model=d_model,
              embedding_dim=embedding_dim,
              num_attention_heads=num_attention_heads,
              dff=dff,
            #   dropout_rate=dropout_rate,
              name=f"encoder_sublayer_{i}")
            for i in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(dropout_rate)

    def compute_mask(self, x, previous_mask=None):
        x = self.tokenizer(x)
        return self.pos_embedding.compute_mask(x, previous_mask)

    def call(self, x, training):
        # Sum up embeddings and positional encoding.
        x = self.tokenizer(x)
        x = self.pos_embedding(x)# Shape `(batch_size, input_seq_len, d_model)`.
        # Add dropout.
        # ?
        # x = self.dropout(x, training=training)

        # N encoder layers.
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training)

        return x  # Shape `(batch_size, input_seq_len, d_model)`.


class SpaceSegmentationTransformer(tf.keras.Model):
    """
    Transformer for finding space in english text without spaces

    Encoder takes in batches of strings, decoder takes in batches of seq_len
    tokens indicating space or not.

    """
    def __init__(self,
               *,
               num_layers, # Number of decoder layers.
               d_model, # Input/output dimensionality.
               num_attention_heads,
               dff, # Inner-layer dimensionality.
               input_tokenizer,
               seq_len,
               embedding_dim,
               dropout_rate=0.1,
               units=20,
               classification_threshold=0.5,
               num_classes=2,
               pos_multiplier=1,
               ):
        super().__init__()
        d_model = d_model + 1 if d_model % 2 == 1 else d_model # Ensure even dimensionality so our positional encodings work
        self.encoder = Encoder(
          num_layers=num_layers,
          d_model=d_model,
          embedding_dim=embedding_dim,
          num_attention_heads=num_attention_heads,
          dff=dff,
          tokenizer=input_tokenizer,
          dropout_rate=dropout_rate,
          seq_len=seq_len,
          pos_multiplier=pos_multiplier
          )
        self.dense = tf.keras.layers.Dense(units)
        self.dense_classify = tf.keras.layers.Dense(1, activation="sigmoid")  # Why does softmax here break it?

    def call(self, inputs, training=False):
        """
        Expects inputs of text to be segmentated and, if training, segmentation
        labels aligned to input text - where 2 indicates space, 1 indicates 
        no space and 0 indicates a missing character (essentially end token).

        Assumes batch first ranks
        """
        to_enc = inputs
        run_sequential = False

        if not training:
            # This is where we will migrate sequential generation once we re-include the decoder
            batch_size = tf.shape(to_enc)[0]
            to_dec = tf.zeros((batch_size, 1))
            run_sequential = True

        
        # The encoder output.
        enc_output = self.encoder(to_enc, training)  # `(batch_size, inp_seq_len, d_model)`
        # enc_mask = self.encoder.compute_mask(to_enc)
        h = self.dense(enc_output)
        # h = enc_output
        return tf.reshape(self.dense_classify(h), [tf.shape(inputs)[0], SEQ_LEN])

# Transformer Model

In [60]:
tokenizer = tf.keras.layers.TextVectorization(
    output_sequence_length=SEQ_LEN,
    standardize="lower_and_strip_punctuation",
    split="character",
    output_mode="int",
)
tokenizer.adapt(train_ds.take(10).map(lambda x,y: x))

In [61]:
tokenizer(test_inputs)

<tf.Tensor: shape=(128, 10), dtype=int64, numpy=
array([[3, 2, 2, ..., 3, 3, 3],
       [3, 2, 3, ..., 3, 3, 3],
       [2, 2, 2, ..., 2, 2, 3],
       ...,
       [2, 2, 3, ..., 2, 2, 2],
       [2, 2, 3, ..., 3, 2, 2],
       [3, 3, 3, ..., 3, 3, 2]])>

In [67]:
NUM_HEADS = 2
KEY_DIM = 16
FF_DIM = 32
DROPOUT_RATE = 0.1
UNITS = 20

transformer = SpaceSegmentationTransformer(
    seq_len=SEQ_LEN,
    num_layers=1,
    embedding_dim=32,
    d_model=KEY_DIM,
    num_attention_heads=NUM_HEADS,
    dff=FF_DIM,
    input_tokenizer=tokenizer,
    units=UNITS,
    dropout_rate=DROPOUT_RATE
)

16 32 2 32


In [68]:
a = EncoderLayer(
    d_model=16,
    embedding_dim=32,
    num_attention_heads=2,
    dff=32
)
B, S, E = 16, 10, 32
ins_ = tf.keras.Input((S,E))
a = Model(ins_, a(ins_, mask=None))
a.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 10, 32)]          0         
                                                                 
 encoder_layer_7 (EncoderLay  (None, 10, 32)           6464      
 er)                                                             
                                                                 
Total params: 6,464
Trainable params: 6,464
Non-trainable params: 0
_________________________________________________________________


In [69]:
# Test

transformer_out = transformer(test_inputs)
transformer_out.shape, transformer.summary()

Model: "space_segmentation_transformer_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_10 (Encoder)        multiple                  6592      
                                                                 
 dense_64 (Dense)            multiple                  660       
                                                                 
 dense_65 (Dense)            multiple                  21        
                                                                 
Total params: 7,273
Trainable params: 7,273
Non-trainable params: 0
_________________________________________________________________


(TensorShape([128, 10]), None)

In [70]:
transformer_out

<tf.Tensor: shape=(128, 10), dtype=float32, numpy=
array([[0.11872023, 0.24394152, 0.26040784, ..., 0.04773878, 0.08857819,
        0.17722492],
       [0.1235707 , 0.2484181 , 0.12467131, ..., 0.04853757, 0.08891717,
        0.17909034],
       [0.2577435 , 0.24241745, 0.25894222, ..., 0.09497575, 0.16285242,
        0.17643547],
       ...,
       [0.2595851 , 0.24322022, 0.12102034, ..., 0.09509356, 0.1636229 ,
        0.31446713],
       [0.25720707, 0.24203947, 0.11997125, ..., 0.04739778, 0.16261807,
        0.3129458 ],
       [0.12326994, 0.11876928, 0.12467419, ..., 0.04849821, 0.08889298,
        0.31986976]], dtype=float32)>

In [71]:
def masked_binary_crossentropy(y_true, y_pred):
  """
  y_true: shape (batch_size, seq_len). 1. = no space, 2. = space
  y_pred: shape (batch_size, seq_len, 1). Logits
  """
  labels = y_true  # 0 = no space, 1 = space

  # Deal with Keras 'feature' that squeezes out the last dimension silently (WTF)
  # if tf.shape(y_pred)[-1] == 1:  
  #   logits = tf.squeeze(y_pred, axis=-1)  # (batch_size, seq_len)
  # else:
  logits = y_pred
  probs = tf.nn.sigmoid(logits)
  bce = - labels * tf.math.log(probs) - ((1 - labels) * tf.math.log(1 - probs))

  return tf.reduce_mean(bce)

In [72]:
# Test

masked_binary_crossentropy(test_outputs, transformer_out)

<tf.Tensor: shape=(), dtype=float32, numpy=0.72823274>

In [73]:
for test_inputs, test_outputs in train_ds.take(1):
  y_pred = transformer(test_inputs)
  loss = masked_binary_crossentropy(test_outputs, y_pred)

loss

<tf.Tensor: shape=(), dtype=float32, numpy=0.7276015>

In [74]:
from tensorflow.keras.metrics import BinaryAccuracy, Precision, Recall, AUC

earlystopping = EarlyStopping(patience=2, monitor="val_binary_accuracy")
transformer.compile(loss=masked_binary_crossentropy, optimizer='adam',
                    metrics=[BinaryAccuracy(), Precision(), Recall(), AUC(curve='PR')])

history = transformer.fit(train_ds, validation_data=valid_ds, epochs=20, callbacks=[earlystopping])

Epoch 1/20
79/79 [==============================] - 5s 32ms/step - loss: 0.6943 - binary_accuracy: 0.7745 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - auc_2: 0.2387 - val_loss: 0.6932 - val_binary_accuracy: 0.7744 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - val_auc_2: 0.2256
Epoch 2/20
79/79 [==============================] - 2s 24ms/step - loss: 0.6932 - binary_accuracy: 0.7745 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - auc_2: 0.2255 - val_loss: 0.6932 - val_binary_accuracy: 0.7744 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - val_auc_2: 0.2256
Epoch 3/20
79/79 [==============================] - 2s 24ms/step - loss: 0.6932 - binary_accuracy: 0.7745 - precision_2: 0.0000e+00 - recall_2: 0.0000e+00 - auc_2: 0.2255 - val_loss: 0.6932 - val_binary_accuracy: 0.7744 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - val_auc_2: 0.2256
Epoch 4/20
79/79 [==============================] - 2s 25ms/step - loss: 0.6932 - binary_accuracy: 0.7745 - p

In [ ]:
transformer.evaluate(test_ds, return_dict=True)

In [ ]:
transformer(['ababab'+'b'*(SEQ_LEN-6)])